# Ejemplo de Inferencia

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install ultralytics

In [1]:
import os
import shutil
import json
from ultralytics import YOLO

## Ajustes Iniciales

In [2]:
# Nombre del alumno
student_name = "leandro_salvañá"

# Ruta al archivo de pesos
model_path = "./Results/yolov8_retraining/weights/best.pt"

# Ruta al directorio que contiene las imagenes
imgs_dir = "./Data/eval/images/val"

# Ruta al directorio de destino de las detecciones
base_dir = "./Results/Envido"
dets_dir = os.path.join(base_dir, student_name)

In [3]:
# Reestablecimiento del directorio de destino (eliminacion)
if os.path.exists(dets_dir):
    shutil.rmtree(dets_dir)

os.makedirs(dets_dir)

## Inferencia

### Formato YOLOv5 para las salidas de las inferencias

Estas son las celdas para que el estudiante complete con su código de inferencia.

Debe generar un archivo con las detecciones en formato YOLOv5, donde cada línea contiene:
 cls, xc, yx, w, h, c

* **cls**: número de índice de la clase

* **xc**: coordenada x al centro de la caja delimitadora (bbox)

* **yx**: coordenada y al centro de la caja delimitadora (bbox)

* **w**: ancho de la caja delimitadora

* **h**: alto de la caja delimitadora

* **c**: certeza del modelo sobre la clasificacíon (confidence)


 Todas las coordenadas deben ser relativas: [0 ... 1]

NOTA: Si utiliza un modelo YoloV8 de ultralytics, pueder llamar al método result.save_txt(file_name, save_conf=True)

### Formato para los archivos de envido.json

Además de las detecciones anteriores, debe existir un archivo llamado envido.json con la siguiente estructura:

```json
{
    "IMG_20240630_174514649_HDR.jpg": {
        "total_cards": 3,
        "cards": {
            "E": [],
            "C": [
                12
            ],
            "B": [
                11
            ],
            "O": [
                10
            ]
        },
        "points": 0,
        "figure": "N/A"
    },
    "IMG_20240630_173918579.jpg": {
        "total_cards": 2,
        "cards": {
            "E": [],
            "C": [],
            "B": [],
            "O": [
                9,
                8
            ]
        },
        "points": 0,
        "figure": "N/A"
    },
  ...
}
```

Esto indica que hay un total de tres cartas, que son el 5 y el 1 de Copas, y el 2 de Basto, que dan 26 puntos de envido de Copa.

Este archivo debe contener una clave con el nombre de archivo, donde cada elemento es a su vez un objeto cons las claves detalladas.

Revise el archivo gt_envido.json para analizar todas las posibilidades.

### DEMO: cálculo del envido

A continuación hay una implementación del cálculo de los puntos a modo de ejemplo, que sirve de referencia para evaluar el algoritmo de evaluación.

**ESTA ES LA SECCIÓN QUE CADA ALUMNO REEMPLAZARIA CON SU CODIGO**

In [4]:
# Cargar el modelo entrenado
model = YOLO(model_path)

# Realizar inferencia sobre todas las imágenes del directorio
image_files = [f for f in os.listdir(imgs_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for image_file in image_files:
    # Obtener el nombre base
    base_name = image_file[:image_file.rfind('.')]
   
    # Crear el nombre para el txt
    txt_name = base_name + '.txt'
    
    # Ruta del txt
    txt_path = os.path.join(dets_dir, txt_name)
    
    # Ruta de la imagen
    image_path = os.path.join(imgs_dir, image_file)

    # Realizar inferencia
    result = model(image_path)
    
    # Guardar los resultados en el directorio de detecciones
    result[0].save_txt(txt_path , save_conf=True)

print(f"Inferencia completada. Resultados guardados en {dets_dir}.")


image 1/1 /home/salvanya/TP-CV2/Data/eval/images/val/IMG_20240630_173929084.jpg: 640x512 1 3O, 1 5O, 1 6O, 1 7O, 195.7ms
Speed: 12.3ms preprocess, 195.7ms inference, 10.0ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /home/salvanya/TP-CV2/Data/eval/images/val/IMG_20240630_173836085.jpg: 640x512 1 6B, 1 7E, 112.8ms
Speed: 3.9ms preprocess, 112.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /home/salvanya/TP-CV2/Data/eval/images/val/IMG_20240630_173828513_HDR.jpg: 640x512 1 3B, 1 6B, 1 7E, 162.1ms
Speed: 5.6ms preprocess, 162.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /home/salvanya/TP-CV2/Data/eval/images/val/IMG_20240630_174434126.jpg: 640x512 1 7C, 1 12O, 159.5ms
Speed: 6.0ms preprocess, 159.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /home/salvanya/TP-CV2/Data/eval/images/val/IMG_20240630_173918579.jpg: 640x512 1 7O, 1 8O, 132.2ms
Speed: 3.0ms preprocess, 132.2ms 

In [5]:
# Lista de clases
classes = [
    '1O', '1C', '1E', '1B', '2O', '2C', '2E', '2B', '3O', '3C', '3E', '3B', 
    '4O', '4C', '4E', '4B', '5O', '5C', '5E', '5B', '6O', '6C', '6E', '6B', 
    '7O', '7C', '7E', '7B', '8O', '8C', '8E', '8B', '9O', '9C', '9E', '9B', 
    '10O', '10C', '10E', '10B', '11O', '11C', '11E', '11B', '12O', '12C', 
    '12E', '12B', 'J'
]

# Obtener lista de imágenes con extensiones válidas
valid_extensions = ['.png', '.jpg', '.jpeg']
image_files = [f for f in os.listdir(imgs_dir) if os.path.splitext(f)[1].lower() in valid_extensions]

# Crear diccionario de imagen base y su extensión
image_dict = {os.path.splitext(f)[0]: f for f in image_files}

# Inicializar diccionario de resultados
results_dict = {}

# Leer archivos .txt en dets_dir
txt_files = [f for f in os.listdir(dets_dir) if f.endswith('.txt')]

for txt_file in txt_files:
    base_name = os.path.splitext(txt_file)[0]
    
    # Verificar si la imagen correspondiente existe
    if base_name in image_dict:
        image_name = image_dict[base_name]
        image_classes = []
        
        # Leer el archivo .txt y obtener las clases
        txt_path = os.path.join(dets_dir, txt_file)
        with open(txt_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                class_id = int(parts[0])
                class_name = classes[class_id]
                image_classes.append(class_name)
        
        # Añadir al diccionario de resultados
        results_dict[image_name] = image_classes

# Eliminar detecciones múltiples
for image_name, classes in results_dict.items():
    results_dict[image_name] = list(set(classes))

print(results_dict)

{'IMG_20240630_173929084.jpg': ['7O', '3O', '6O', '5O'], 'IMG_20240630_174418639.jpg': ['6O', '10O'], 'IMG_20240630_174223161_HDR.jpg': ['9O', '12B'], 'IMG_20240630_174246178_HDR.jpg': ['2C'], 'IMG_20240630_174235296.jpg': ['4O', '8C', '10E'], 'IMG_20240630_173822184_HDR.jpg': ['9B', '7E'], 'IMG_20240630_174434126.jpg': ['7C', '12O'], 'IMG_20240630_174133804_HDR.jpg': ['4E', '5E', '3E'], 'IMG_20240630_173918579.jpg': ['7O', '8O'], 'IMG_20240630_174155383_HDR.jpg': ['12E'], 'IMG_20240630_174115784.jpg': ['8C', '9E'], 'IMG_20240630_174453153.jpg': ['3C', '1B'], 'IMG_20240630_173758691.jpg': ['1C', '2B'], 'IMG_20240630_174514649_HDR.jpg': ['12C', '11B', '12O'], 'IMG_20240630_174333447.jpg': ['7B', '1B'], 'IMG_20240630_174349395.jpg': ['10B', '11B', '7O'], 'IMG_20240630_173828513_HDR.jpg': ['3B', '6B', '7E'], 'IMG_20240630_174302382_HDR.jpg': ['6C', '1E'], 'IMG_20240630_174005265.jpg': ['1O'], 'IMG_20240630_173711753_HDR.jpg': ['11C'], 'IMG_20240630_173859456.jpg': ['3B'], 'IMG_20240630_17

In [7]:
# Puntajes de las cartas
points_dict = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: 7,
    8: 0,
    9: 0,
    10: 0,
    11: 0,
    12: 0
}

# Función para calcular puntos y palo
def calculate_points_and_figure(cards_dict):
    counts = {key: len(cards_dict[key]) for key in cards_dict}
    max_point = 0
    max_figure = "N/A"
    
    # Filtrar clases con más de una carta y sumar puntos
    for key, values in cards_dict.items():
        if len(values) > 0:
            # Obtener máximo puntaje y figura
            max_card = max(values)
            if points_dict[max_card] > max_point:
                max_point = points_dict[max_card]
                max_figure = key
            
            # Sumar puntos de cartas válidas (no 8 o 9)
            valid_points = [points_dict[card] for card in values if points_dict[card] > 0]
            if len(valid_points) > 1:
                max_point = max(max_point, sum(sorted(valid_points)[-2:]) + 20)
                max_figure = key
    
    return max_point, max_figure

# Construir el nuevo diccionario
card_ds_file = {}

for image_name, classes in results_dict.items():
    # Inicializar estructura
    card_entry = {
        "total_cards": len(classes),
        "cards": {
            "E": [],
            "C": [],
            "B": [],
            "O": []
        },
        "points": 0,
        "figure": "N/A"
    }
    
    # Descomponer las clases en el diccionario de cartas
    for class_label in classes:
        number = int(class_label[:-1])
        letter = class_label[-1]
        card_entry["cards"][letter].append(number)
    
    # Calcular puntos y figura
    points, figure = calculate_points_and_figure(card_entry["cards"])
    card_entry["points"] = points
    card_entry["figure"] = figure
    
    # Añadir entrada al diccionario final
    card_ds_file[image_name] = card_entry

print(card_ds_file)


{'IMG_20240630_173929084.jpg': {'total_cards': 4, 'cards': {'E': [], 'C': [], 'B': [], 'O': [7, 3, 6, 5]}, 'points': 33, 'figure': 'O'}, 'IMG_20240630_174418639.jpg': {'total_cards': 2, 'cards': {'E': [], 'C': [], 'B': [], 'O': [6, 10]}, 'points': 0, 'figure': 'N/A'}, 'IMG_20240630_174223161_HDR.jpg': {'total_cards': 2, 'cards': {'E': [], 'C': [], 'B': [12], 'O': [9]}, 'points': 0, 'figure': 'N/A'}, 'IMG_20240630_174246178_HDR.jpg': {'total_cards': 1, 'cards': {'E': [], 'C': [2], 'B': [], 'O': []}, 'points': 2, 'figure': 'C'}, 'IMG_20240630_174235296.jpg': {'total_cards': 3, 'cards': {'E': [10], 'C': [8], 'B': [], 'O': [4]}, 'points': 4, 'figure': 'O'}, 'IMG_20240630_173822184_HDR.jpg': {'total_cards': 2, 'cards': {'E': [7], 'C': [], 'B': [9], 'O': []}, 'points': 7, 'figure': 'E'}, 'IMG_20240630_174434126.jpg': {'total_cards': 2, 'cards': {'E': [], 'C': [7], 'B': [], 'O': [12]}, 'points': 7, 'figure': 'C'}, 'IMG_20240630_174133804_HDR.jpg': {'total_cards': 3, 'cards': {'E': [4, 5, 3], 

## Escritura del archivo envido.json

In [8]:
with open(os.path.join(dets_dir, "envido.json"), "w") as jf:
    json.dump(card_ds_file, jf, indent=4)

## Escritura de archivo gt_envido.json 

Esta celda responde a una confusión con el README.txt. Porque la celda anterior, que ya venía escrita, hace que el JSON se guarde en dets_dir, es decir, el directorio que definimos al principio. Pero en el README.txt dice que en el archivo "gt_envido.json" tiene que estar en la misma carpeta que el dataset.yaml, que a su vez tiene que tener dos directorios uno con las imágenes y otra con las etiquetas. No sé si ese JSON el que ya viene con el código de evaluación provisto o el que tengo que crear yo. Porque el código que vino escrito en la celda anterior, escribe un archivo "envido.json" no un "gt_envido.json", y además lo hace en un directorio, a priori, independiente.

Entonces, de ser incorrecta la escritura del json anterior, descomentar y correr la siguiente celda para que el json que se escriba lleve el nombre "gt_envido.json" y esté en el mismo directorio que el yaml.

In [27]:
# with open(os.path.join('data/eval', "gt_envido.json"), "w") as jf:
#     json.dump(card_ds_file, jf, indent=4)